# Chat Testing

Here I am going to be figuring out all of the different aspects of how the chatbot will work.

In [23]:
from openai import OpenAI
import json
import pg8000

In [24]:
# Load Chat API Key and DataBase Data
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path="./.env")

# Access the OPENAI_API_KEY variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Basic Load of OpenAI

In [25]:
client = OpenAI(api_key=OPENAI_API_KEY)

try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {
              "role": "user",
              "content": "Write a haiku about recursion in programming."
          }
      ]
  )

  print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

KeyboardInterrupt: 

### Differentiate between Recipe, General, or Lifestyle

In [ ]:
user_question = "Why do you say that?"

user_diet = "celiac diet and low fodmap diet"

rules = """
1. You are the first step in a longer chatbot which has the goal of helping users
with specific dietary needs to answer general questions about the dietary needs, provide lifestyle advice 
in the context of said dietary need, or provide recipes specific to the user's needs.
2. Your job is only to differentiate the requests, so the next request to the chatbot will be able to have
a specific format.
For example, recipes will be returned in a different format from lifestyle advice, which will be
different from general queries.
3. Your job is to differentiate between general queries, lifestyle tips, and recipe requests.
4. If someone is asking for a recipe, you will respond with only 'recipe'.
5. If someone is asking general questions about their dietary needs, you will answer with "general"
6. If someone is asking about things that will be useful general knowledge living with their dietary need, you will answer "lifestyle"
7. Anything the user asks which does not pertain their diet, a diet, or the lifestyle associated with that diet, or claryfying questions, answer "not applicable"
"""


try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": f"{rules}"},
          {"role": "assistant", "content": f"{user_diet}"},
          {
              "role": "user",
              "content": f"{user_question}"
          }
      ]
  )

  print(completion.choices[0].message.content)
  query = completion.choices[0].message.content
except Exception as e:
    print(f"An error occurred: {e}")


not applicable


### Recipe Responses

In [ ]:
# Create the chat completion request for recipes
response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give recipes given certain dietary conditions. 
            You must make sure the recipe is not repeated.
            You must make sure the recipe does not recommend anything the user cannot eat
            with their dietary restrictions"""
        },
        {
            "role": "assistant",
            "content": "Dietary restrictions: User has the following di"
        },
        {
            "role": "user",
            "content": "I would like a basic recipe with chicken and vegetables which takes less than 30 minutes."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recipe_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "Recipe": {
                        "description": "The name of the recipe",
                        "type": "string"
                    },
                    "Ingredients": {
                        "description": "list of ingredients with measurements if needed",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "Prep Time": {
                        "description": "time in minutes to prepare this recipe",
                        "type": "integer"
                    },
                    "Instructions": {
                    "description": "Instructions to create this recipe, stored as JSON",
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                },
                "additionalProperties": False
            }
        }
    }
)

# Print the response
print(response.choices[0].message.content)
recipe = json.loads(response.choices[0].message.content)

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' on line 21 (2988229417.py, line 54)

In [ ]:
recipe['Recipe']

'Quick Chicken and Vegetable Stir-Fry'

In [41]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()

# # Fetch table names from Database
# cursor.execute("""
#     SELECT table_name 
#     FROM information_schema.tables 
#     WHERE table_schema = 'public';
# """)
# tables = cursor.fetchall()
# print("Tables in the database:", [table[0] for table in tables])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'users';
# """)
# columns = cursor.fetchall()
# print("Columns in 'users' table:", [col[0] for col in columns])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'userpreference';
# """)
# columns = cursor.fetchall()
# print("Columns in 'userpreference' table:", [col[0] for col in columns])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'diettype';
# """)
# columns = cursor.fetchall()
# print("Columns in 'diettype' table:", [col[0] for col in columns])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'context';
# """)
# columns = cursor.fetchall()
# print("Columns in 'context' table:", [col[0] for col in columns])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'tip';
# """)
# columns = cursor.fetchall()
# print("Columns in 'tip' table:", [col[0] for col in columns])

# # Fetch Column names from user table
# cursor.execute("""
#     SELECT column_name 
#     FROM information_schema.columns 
#     WHERE table_name = 'recipe';
# """)
# columns = cursor.fetchall()
# print("Columns in 'recipe' table:", [col[0] for col in columns])

# Example query: Fetch User table
cursor.execute("SELECT * FROM context;")
rows = cursor.fetchall()
# print(rows)



# Close the connection
conn.close()

# Return fetched context as a dictionary
# return {key: value for key, value in rows}

rows

([1,
  1,
  'Can I eat Broccoli?',
  'Yes, but only steamed',
  None,
  None,
  datetime.datetime(2024, 11, 18, 10, 0)],
 [2,
  1,
  'I want a recipe with beef',
  'Beef and Vegetable Stir-Fry',
  16,
  None,
  datetime.datetime(2024, 11, 21, 20, 35, 30, 557000)],
 [3,
  1,
  'this is a test',
  'Test',
  None,
  None,
  datetime.datetime(2024, 11, 21, 20, 47, 25, 270000)],
 [4,
  1,
  'I want a recipe with beef',
  'Beef and Sweet Potato Skillet',
  20,
  None,
  datetime.datetime(2024, 11, 22, 19, 57, 13, 260000)],
 [5,
  1,
  'I want a recipe with beef',
  'Beef and Quinoa Bowls',
  21,
  None,
  datetime.datetime(2024, 11, 22, 20, 2, 50, 675000)],
 [6,
  1,
  'I want a recipe with beef',
  'Beef and Vegetable Soup',
  22,
  None,
  datetime.datetime(2024, 11, 22, 20, 4, 32, 359000)],
 [7,
  1,
  'I want a recipe with beef',
  'Beef and Spinach Stuffed Peppers',
  31,
  None,
  datetime.datetime(2024, 11, 22, 20, 21, 6, 682000)],
 [8,
  1,
  'I want a recipe with beef',
  'Beef and 

In [37]:
def get_conversation_context(uid, cursor):
    """ 
    Gets context to give to chatbot 
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    List of 10 most previous questions and answers formatted to give
    chatbot correct context
    """

    try:
        query = f"""
            SELECT "userQuestion", "chatResponse"
            FROM context
            WHERE uid = {uid} AND "recipeId" IS NULL
            ORDER BY timestamp DESC
            LIMIT 10;
        """
        cursor.execute(query)
        context = cursor.fetchall()
        # Flatten the context into a list of messages
        messages = []
        for user_q, chat_resp in reversed(context):
            messages.append({"role": "user", "content": user_q})
            messages.append({"role": "assistant", "content": chat_resp})
        return messages
    except Exception as e:
        print(f"Database error: {e}")
        return []

In [38]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()


context = get_conversation_context(1, cursor)


conn.close()
context

[{'role': 'user', 'content': 'Can I eat Broccoli?'},
 {'role': 'assistant', 'content': 'Yes, but only steamed'},
 {'role': 'user', 'content': 'this is a test'},
 {'role': 'assistant', 'content': 'Test'}]